# Advanced programming and data analysis

## HSE University, 2024-25 Academic Year

# Practice 1. Introduction to sklearn. KNN

### General workflow for solving a machine learning problem

__Recall from the lecture:__
* What is a machine learning task? What is given and what should be found?
* What types of features are there in machine learning?
* What types of tasks are there in machine learning?
* What is a loss/quality function? What is it used for?

Let us recall the general scheme for solving a machine learning problem:

<div>
<img src="https://github.com/hse-ds/iad-intro-ds/blob/master/2023/seminars/sem05_sklearn_knn/static/scheme.png?raw=true" width="500"/>
</div>

From the initial database, after preprocessing, we obtain a training sample $X, Y$.

The object-attribute matrix $X$ has the size (number of objects) $\times$ (number of features). One row of this matrix corresponds to one object of the training sample given as a vector of length (number of features). The features are numerical characteristics of the object. The vector of correct answers $Y$ has length (number of objects).

At the training stage, an algorithm $a(x)$ is built (trained) on the basis of the training sample $X, Y$. It is a function that takes as an input the features of an object and returns a prediction for that object: $y \approx a(x)$. Algorithm $a$ can make predictions for any valid objects; it can be applied to both learning objects and objects that the algorithm has never seen. This is the goal of machine learning: to identify patterns in the training sample that will allow one to make high-quality (fairly accurate) predictions on new objects $x$.

How to train such algorithms $a(x)$ on a training sample is largely the focus of our course.


### Scikit-learn Interface

Scikit-Learn, or Sklearn for short, is a library that implements almost all machine learning algorithms used today. We need to get familiar with the interface of the library to understand how it can be used in practice. Further in the course we will not only use ready-made implementations from sklearn, but sometimes we will implement algorithms ourselves in the same spirit as in this library (with the same interface).

To implement machine learning algorithms in sklearn, one interface is always used - a class with the `fit(X, Y)` method for training the model on the training sample $X, Y$ and predict(X) for returning predictions on the sample $X$. When creating a class, you can specify additional parameters that affect the operation of the machine learning algorithm.

For example, this would be the logic of the linear regression class, which we will explore in detail during the next seminars:

* When creating the class, we need to remember the regularization coefficient;
* The purpose of the `fit` function is to find the weights w from the sample $X$ and $Y$ and store them inside the class as self.w;
* The purpose of the `predict` function is to return the predictions of $Y$ according to the weights of self.w and $X$.


$$w = (X^TX + \lambda E)^{-1}X^Ty$$

### Seminar Agenda
- Data preprocessing using scikit-learn
        - handling missing values
        - <handling outliers>
        - converting categorical features to numeric
        - feature scaling
- Distance-based methods. k Nearest Neighbors
        - Theory part
        - Practical part

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
class LinearRegressor:
    def __init__(self, reg_coef: float = None) -> None:
        self.lambda_ = reg_coef

    def fit(self, X_train: np.array, y_train: np.array) -> None:
        self.w = ...

    def predict(self, X_test: np.array) -> np.array:
        y_pred = ...

        return y_pred

If we didn't use a class, we would have to pass the weights `w` to the `predict` function every time we wanted to make predictions, whereas this way they are stored inside the class. This is especially useful if there are many such auxiliary variables.

In addition to training and prediction algorithms for different methods, sklearn implements a lot of auxiliary functionality for data preprocessing, data visualization, computing quality metrics, etc. In the following seminars, we will gradually get to know this functionality of the library.

Today we will learn about data preprocessing methods and their implementation in sklearn. For demonstrations, we will load the [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/Automobile). The data contains categorical, integer and real-valued attributes.

In [ ]:
X_raw = pd.read_csv(
    "http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",
    header=None,
    na_values=["?"],
)
X_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


Let's separate the attributes and the target variable:

In [ ]:
y = X_raw[25]
X_raw = X_raw.drop(25, axis=1)

## Data Preparation

### Filling the gaps
In a feature matrix, there may be missing values, which can cause an error when passing the matrix to a model training function or even to a preprocessing step. If there are only a few missing values, one option is to remove the corresponding samples from the training set. Alternatively, the missing values can be filled in using various techniques:

- filling with summary statistics (mean or median);
- predicting missing values based on the observed features.


The second approach is more complex and less commonly used. For filling with constant values, you can use the fillna method of a DataFrame. To replace missing values with the mean or other statistics, use the  `impute.SimpleImputer`   class



In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
# To make working with our dataset easier, we create a mask that identifies the columns with categorical features.
# Categorical features typically have the data type "object"
cat_features_mask = (X_raw.dtypes == "object").values

# For numerical features, we'll fill in the missing values using the mean.
X_real = X_raw[X_raw.columns[~cat_features_mask]]
mis_replacer = SimpleImputer(strategy="mean")
X_no_mis_real = pd.DataFrame(
    data=mis_replacer.fit_transform(X_real), columns=X_real.columns
)

# For categorical features, we'll fill in the missing values with empty strings.
X_cat = X_raw[X_raw.columns[cat_features_mask]].fillna("")
X_no_mis = pd.concat([X_no_mis_real, X_cat], axis=1)

X_no_mis.head()

,0,1,9,10,11,12,13,16,18,19,...,2,3,4,5,6,7,8,14,15,17
0,3.0,122.0,88.6,168.8,64.1,48.8,2548.0,130.0,3.47,2.68,...,alfa-romero,gas,std,two,convertible,rwd,front,dohc,four,mpfi
1,3.0,122.0,88.6,168.8,64.1,48.8,2548.0,130.0,3.47,2.68,...,alfa-romero,gas,std,two,convertible,rwd,front,dohc,four,mpfi
2,1.0,122.0,94.5,171.2,65.5,52.4,2823.0,152.0,2.68,3.47,...,alfa-romero,gas,std,two,hatchback,rwd,front,ohcv,six,mpfi
3,2.0,164.0,99.8,176.6,66.2,54.3,2337.0,109.0,3.19,3.40,...,audi,gas,std,four,sedan,fwd,front,ohc,four,mpfi
4,2.0,164.0,99.4,176.6,66.4,54.3,2824.0,136.0,3.19,3.40,...,audi,gas,std,four,sedan,4wd,front,ohc,five,mpfi


It’s always important to analyze whether the missing values in a feature occur randomly. Sometimes, the very fact that information is missing can be a valuable signal that should be turned into a new feature.

__Example:__ predicting a user’s age based on data from their phone. Older users tend to use simpler phones, so the absence of certain data (like browsing history) may actually be a strong indicator of age.

For categorical features, it's often recommended to introduce a separate category to represent missing values. In our case, there are no missing values in the categorical features.

### Converting categorical features to numerical format
Almost all machine learning algorithms require the input to the training function to be a numerical (float) matrix. During training, the algorithms rely on properties of real numbers — such as the ability to compare values and perform arithmetic operations. Therefore, even if the feature matrix contains numeric-looking values, it's important to assess whether they should actually be treated as numerical features.

__Example:__ Some features may be represented as integer hashes or IDs (for example, a social media user ID). However, you can’t add two users and get a third one based on their IDs — but that’s exactly the kind of operation a linear model might try to perform if it treats IDs as numerical features.

This is an example of a categorical feature that takes values from an unordered, finite set $K$. A common approach for such features is to apply [one-hot encoding](http://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features), which replaces the original feature with $K$ binary features — one for each possible category value. In `sklearn`, this can be done using the `LabelEncoder` and `OneHotEncoder` classes, but it's often easier to use the `pd.get_dummies` function.

Note that the resulting matrix will contain a lot of zeros. To avoid storing them in memory, you can set the parameter `OneHotEncoder` or `.get_dummies`, and the method will return a [sparse matrix](http://docs.scipy.org/doc/scipy/reference/sparse.html), which only stores the non-zero values. While some operations on sparse matrices can be inefficient, most `sklearn` methods support working with sparse input.



In [ ]:
print(f"Shape before encoding: {X_no_mis.shape}")
X_dum = pd.get_dummies(X_no_mis, drop_first=True)
X_dum

Shape before encoding: (205, 25)


,0,1,9,10,11,12,13,16,18,19,...,15_three,15_twelve,15_two,17_2bbl,17_4bbl,17_idi,17_mfi,17_mpfi,17_spdi,17_spfi
0,3.0,122.0,88.6,168.8,64.1,48.8,2548.0,130.0,3.47,2.68,...,0,0,0,0,0,0,0,1,0,0
1,3.0,122.0,88.6,168.8,64.1,48.8,2548.0,130.0,3.47,2.68,...,0,0,0,0,0,0,0,1,0,0
2,1.0,122.0,94.5,171.2,65.5,52.4,2823.0,152.0,2.68,3.47,...,0,0,0,0,0,0,0,1,0,0
3,2.0,164.0,99.8,176.6,66.2,54.3,2337.0,109.0,3.19,3.40,...,0,0,0,0,0,0,0,1,0,0
4,2.0,164.0,99.4,176.6,66.4,54.3,2824.0,136.0,3.19,3.40,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1.0,95.0,109.1,188.8,68.9,55.5,2952.0,141.0,3.78,3.15,...,0,0,0,0,0,0,0,1,0,0
201,-1.0,95.0,109.1,188.8,68.8,55.5,3049.0,141.0,3.78,3.15,...,0,0,0,0,0,0,0,1,0,0
202,-1.0,95.0,109.1,188.8,68.9,55.5,3012.0,173.0,3.58,2.87,...,0,0,0,0,0,0,0,1,0,0
203,-1.0,95.0,109.1,188.8,68.9,55.5,3217.0,145.0,3.01,3.40,...,0,0,0,0,0,1,0,0,0,0


In addition to categorical features, string features also require transformation. They can be converted into a word frequency matrix using [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer), into a matrix of fixed-length character n-gram frequencies, or used to extract other features — such as the length of the string.

### Feature scaling
When working with data, it's always recommended to bring all features to the same scale. This is important for numerical stability when dealing with the feature matrix, since floating-point numbers are more densely distributed near zero than in the range of large values. Additionally, many machine learning algorithms have specific requirements that make feature scaling necessary. For example, in linear models, scaling helps speed up training and improves model interpretability.

One popular scaling method is normalization: subtracting the mean and dividing by the standard deviation for each feature (`StandardScaler` in `sklearn`). Another common approach is min-max scaling: subtracting the minimum and dividing by the range (max - min) of each feature (`MinMaxScaler` in `sklearn`).

In [ ]:
from sklearn import preprocessing

normalizer = preprocessing.MinMaxScaler()
X_real_norm_np = normalizer.fit_transform(X_dum)
X = pd.DataFrame(data=X_real_norm_np)
X.head()

/usr/local/anaconda3/envs/ml/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/anaconda3/envs/ml/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
0,1.0,0.298429,0.058309,0.413433,0.316667,0.083333,0.411171,0.260377,0.664286,0.290476,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.298429,0.058309,0.413433,0.316667,0.083333,0.411171,0.260377,0.664286,0.290476,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.6,0.298429,0.230321,0.449254,0.433333,0.383333,0.517843,0.343396,0.100000,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.8,0.518325,0.384840,0.529851,0.491667,0.541667,0.329325,0.181132,0.464286,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.8,0.518325,0.373178,0.529851,0.508333,0.541667,0.518231,0.283019,0.464286,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Realization Example

Let's implement a class for data normalization, following the interface style of `sklearn`.

In `sklearn`, data preprocessing follows a pattern similar to model training: the `.fit(X)` function stores internal parameters, and `.transform(X)` performs the transformation on the dataset. The target variable `y` is not needed here, since normalization does not involve the target — as is the case with most preprocessing steps.

This class doesn’t require any parameters, so we can skip the `__init__` method. The `.fit()` function calculates statistics — the mean and standard deviation of each feature (based on the training data), and the `.transform()` function subtracts the mean and divides by the standard deviation. We’ll use NumPy to compute the statistics.

In [ ]:
class Normalizer:
    def fit(self, X: np.array) -> None:
        self.mu = X.mean(axis=0)
        self.sigma = X.std(axis=0)

    def transform(self, X: np.array) -> np.array:
        return (X - self.mu[np.newaxis, :]) / self.sigma[np.newaxis, :]

Let’s generate some random data `X` and `y` to test our class:  

In [ ]:
num_obj_train = 20
num_obj_te = 10
num_feat = 4
X_train = np.random.randint(-5, 5, size=(num_obj_train, num_feat))
X_train.shape

(20, 4)

In [ ]:
X_test = np.random.randint(-5, 5, size=(num_obj_te, num_feat))
X_test.shape

(10, 4)

In [ ]:
X_train

array([[ 0,  2,  2, -3],
       [-2,  4,  4, -3],
       [-5, -2, -3, -3],
       [-5,  3, -1,  4],
       [ 4, -3,  3, -5],
       [-5,  1,  4,  2],
       [-1, -1, -5, -1],
       [-2, -1, -1,  1],
       [ 0,  4, -3,  0],
       [ 4, -4,  0,  0],
       [-3, -3,  4, -3],
       [ 1,  0,  2, -5],
       [-5,  2,  1,  2],
       [-3, -1, -3,  4],
       [-1, -1, -3, -4],
       [-1,  3, -4,  0],
       [-4,  4, -4,  0],
       [-4,  3, -1,  1],
       [-1,  0, -2,  2],
       [ 3,  1,  4,  1]])

We create an instance of the class and transform the dataset:

In [ ]:
normalizer = Normalizer()
normalizer.fit(X_train)
X_train_transformed = normalizer.transform(X_train)
X_test_transformed = normalizer.transform(X_test)

The `fit` method should be called specifically on the training data to avoid leaking information from the validation set. The `transform` method, on the other hand, can be called multiple times on any dataset, using the statistics already computed and stored inside the class.

In [ ]:
X_train_transformed

array([[ 0.53199518,  0.58963067,  0.7662411 , -0.92847669],
       [-0.17733173,  1.40291435,  1.4325377 , -0.92847669],
       [-1.24132208, -1.03693669, -0.89950042, -0.92847669],
       [-1.24132208,  0.99627251, -0.23320381,  1.67125804],
       [ 1.95064898, -1.44357853,  1.0993894 , -1.67125804],
       [-1.24132208,  0.18298883,  1.4325377 ,  0.92847669],
       [ 0.17733173, -0.63029485, -1.56579702, -0.18569534],
       [-0.17733173, -0.63029485, -0.23320381,  0.55708601],
       [ 0.53199518,  1.40291435, -0.89950042,  0.18569534],
       [ 1.95064898, -1.85022037,  0.09994449,  0.18569534],
       [-0.53199518, -1.44357853,  1.4325377 , -0.92847669],
       [ 0.88665863, -0.22365301,  0.7662411 , -1.67125804],
       [-1.24132208,  0.58963067,  0.43309279,  0.92847669],
       [-0.53199518, -0.63029485, -0.89950042,  1.67125804],
       [ 0.17733173, -0.63029485, -0.89950042, -1.29986737],
       [ 0.17733173,  0.99627251, -1.23264872,  0.18569534],
       [-0.88665863,  1.

## Distance-based methods: k Nearest Neighbours (k-NN)

### Theoretical Overview

__Recall from the last year:__  
* How are predictions made in the k Nearest Neighbours method for classification and regression tasks?  
* What is the compactness hypothesis?  
* What distance functions can be used for numerical features, categorical features, string features, and multi-valued features?

#### Task 1.  
Suppose we are solving a 3-class classification problem using two features and the k Nearest Neighbours method with *k = 3* and the Manhattan distance metric. We have the following training dataset:

| Feature 1 | Feature 2 | Class |
|-----------|-----------|-------|
| 1         | -1        | 1     |
| 2         | 2         | 1     |
| 3         | 2         | 2     |
| 1         | 0         | 3     |
| 2         | -2        | 3     |

What would the prediction be for the object $x = (2, -1)$?

__Solution.__

The prediction algorithm for kNN in a classification task:  
1. Compute the distance from each training sample to the test object.  
2. Find the *k* training samples (neighbors) with the smallest distances to the test object.  
3. Return the most frequently occurring class among the *k* neighbors.

Let's compute the distances. The distance from the first training sample to the test object $x$ using the Manhattan metric is:

$$|1 - 2| + |-1 - (-1)| = 1.$$

Similarly, for samples 2 to 5, we get distances of 3, 4, 2, and 1, respectively.

The 3 nearest neighbors are samples 1, 4, and 5 (with distances 1, 2, and 1). These correspond to classes 1, 3, and 3. The most frequent class among them is 3, so the predicted class is **3**.

#### Task 2.  
Visualize the decision boundary between classes for the following dataset:

| Feature 1 | Feature 2 | Class |
|-----------|-----------|-------|
| 2         | 2         | 1     |
| 3         | 2         | 1     |
| 2         | 0         | 2     |
| 1         | -1        | 3     |
| 1         | 1         | 3     |

Use *k = 1* and the Euclidean distance.

__Solution.__

In classification problems with two features, we can visualize the feature space as a 2D plane and color it based on the predicted class for each point. That’s exactly our task here.

First, let's plot the training samples — five points — on the plane according to their coordinates.

With $k = 1$, each point in the plane is assigned the same class as its nearest neighbor from the training set. When we have two points from different classes, the class boundary between them is defined by the perpendicular bisector. For multiple points, we construct several such bisectors, find their intersections, and determine which regions correspond to which class. This structure is formally known as a [Voronoi diagram](https://en.wikipedia.org/wiki/Voronoi_diagram), although we won’t go into its mathematical details here.

<div>
<img src="https://github.com/hse-ds/iad-intro-ds/blob/master/2023/seminars/sem05_sklearn_knn/static/classifi.png?raw=true" width="350"/>
</div>

#### Task 3.  
Suppose we are solving a regression problem with two features using the k Nearest Neighbours method with *k = 3* and the Manhattan distance metric. We have the following training dataset:

| Feature 1 | Feature 2 | Target |
|-----------|-----------|--------|
| 1         | -1        | 3.5    |
| 2         | 2         | 2.3    |
| 3         | 2         | 1.7    |
| 1         | 0         | -0.4   |
| 2         | -2        | 0.1    |

What would the prediction be for the object $x = (2, -1)$?

__Solution.__  
The prediction process for kNN in regression differs from classification only in the final step: instead of taking the most frequent class, we average the target values of the nearest neighbors.  
The features in this task are the same as in Task 1, so we already know the nearest neighbors: samples 1, 4, and 5. Their target values are 3.5, -0.4, and 0.1.  

Let’s compute the average:  
$$(3.5 - 0.4 + 0.1)/3 = 1.1$$  
So, the predicted value is **1.1**.

#### Question: What are the parameters and hyperparameters of the kNN method?

__Answer:__

**Parameters** are values adjusted during training using the training data. The kNN algorithm doesn’t involve actual training — it’s a very simple heuristic method. In this context, the parameters of kNN can be interpreted as the training dataset itself. In another interpretation, the method has no parameters at all.

**Hyperparameters** are values that must be set before training the model. They are not learned from the training data. The two most important hyperparameters of the kNN method are the number of neighbors *k* and the distance metric. Different combinations of these hyperparameters can result in very different performance. Hyperparameters are typically tuned using a validation set or cross-validation.


#### How does kNN performance change as *k* increases?

__Answer:__

When $k=1$, each training sample defines its own class region. If, for example, a single noisy sample of a different class ends up in a large region of another class, there will be an "island" of incorrect predictions around this noisy point. This is illogical and indicates overfitting.

When $k$ equals the number of training samples, the model predicts the same class for all points — again, this results in poor classifier performance. This means that kNN accuracy usually increases at first as $k$ grows, then decreases, with the optimal value somewhere in between.

Consider a synthetic example: the training set is visualized below (the “true” decision boundary is a straight line), along with kNN decision boundaries for different values of $k$ — similar to Task 2:

<div>
<img src="https://github.com/hse-ds/iad-intro-ds/blob/master/2023/seminars/sem05_sklearn_knn/static/k_grid.png?raw=true" width="550"/>
</div>

With small $k$, the decision boundary is too complex and heavily influenced by noise. As $k$ increases, the boundary becomes smoother, and at $k=50$ it looks the most reasonable. For even larger $k$, the boundary drifts away from linear, and the orange class starts to “invade” the blue region.

#### Why is it important to normalize data when using kNN?

__Answer:__

Let’s take the Manhattan distance as an example. If one feature has values on the scale of around 1000, while another is around 1, then when we compute the sum of absolute differences, the second feature will have almost no influence on the result. However, if we normalize the features, they will all be on the same scale and contribute equally to the distance calculation.


### Practical Part

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle

In [ ]:
data = load_digits()
X = data.images
y = data.target

X.shape

(1797, 8, 8)

In [ ]:
# We flatten each square image into a vector to obtain a feature matrix (samples × features).
X = X.reshape(X.shape[0], -1)

# We shuffle the data.
X, y = shuffle(X, y)
print(f"Features shape: {X.shape},\nTarget shape: {y.shape}")
print(f"Target samples: {y[:10]}")

Features shape: (1797, 64),
Target shape: (1797,)
Target samples: [0 6 3 4 5 8 0 0 0 1]


In [ ]:
X_train, y_train = X[:700, :], y[:700]
X_val, y_val = X[700:1300, :], y[700:1300]
X_test, y_test = X[1300:, :], y[1300:]

In [ ]:
# We train the classifier and make predictions.
clf = KNeighborsClassifier(n_neighbors=3, p=1, n_jobs=10)

clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)

In [ ]:
# We compute the simplest quality metric for the algorithm — the accuracy
print("Accuracy is: ", np.mean(y_test == y_predicted))

Accuracy is:  0.9859154929577465


Given that we have 10 classes, the chance of randomly guessing the correct label multiple times is very low. So the obtained accuracy is actually a very good result!

Now let’s try using different values of the hyperparameter *k*.  
Comparing *k* values on the training set is pointless: each sample is its own nearest neighbor, so the optimal *k* would always be 1.  
Instead, we’ll compare different *k* values based on validation set performance:

In [ ]:
# Tuning k using the validation set:
k_best = -1
best_accuracy = 0

for k in range(1, 20):
    y_predicted = (
        KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train).predict(X_val)
    )

    val_accuracy = np.mean(y_predicted == y_val)
    print(f"k = {k}; accuracy = {val_accuracy:.3f}")

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        k_best = k

k_best

k = 1; accuracy = 0.975
k = 2; accuracy = 0.967
k = 3; accuracy = 0.977
k = 4; accuracy = 0.968
k = 5; accuracy = 0.972
k = 6; accuracy = 0.968
k = 7; accuracy = 0.967
k = 8; accuracy = 0.965
k = 9; accuracy = 0.963
k = 10; accuracy = 0.963
k = 11; accuracy = 0.968
k = 12; accuracy = 0.958
k = 13; accuracy = 0.957
k = 14; accuracy = 0.958
k = 15; accuracy = 0.958
k = 16; accuracy = 0.957
k = 17; accuracy = 0.952
k = 18; accuracy = 0.953
k = 19; accuracy = 0.947


3

Let’s compare the accuracy on the training, validation, and test sets using the best *k* value we found:

In [ ]:
clf = KNeighborsClassifier(n_neighbors=k_best)
clf.fit(X_train, y_train)

for X_data, y_data in zip([X_train, X_val, X_test], [y_train, y_val, y_test]):
    y_predicted = clf.predict(X_data)
    print(f"Accuracy: {np.mean(y_predicted==y_data):.3f}")

Accuracy: 0.993
Accuracy: 0.977
Accuracy: 0.990


Training set accuracy is the highest, but it's misleading — the algorithm has already seen these samples. On the validation set, we also used the true labels to tune the hyperparameter *k*, so this metric isn’t entirely fair either. In fact, the accuracy on the test set — where we haven’t used the labels at all — may turn out to be lower than on the validation set.

**Conclusion:**  
To fairly evaluate model performance, we should use a **hold-out test set** that is not involved in training or hyperparameter tuning in any way.